In [ ]:
# prompt: upload file

from google.colab import files
uploaded = files.upload()

Saving pizza_sale.xlsx to pizza_sale.xlsx


In [ ]:
# prompt: show data in pizza_sale.xlsx file from "orders" sheet

!pip install openpyxl

import pandas as pd
import io

df = pd.read_excel(io.BytesIO(uploaded['pizza_sale.xlsx']), sheet_name='orders')
df


,order_id,date,time
0,1,2015-01-01,11:38:36
1,2,2015-01-01,11:57:40
2,3,2015-01-01,12:12:28
3,4,2015-01-01,12:16:31
4,5,2015-01-01,12:21:30
...,...,...,...
21345,21346,2015-12-31,20:51:07
21346,21347,2015-12-31,21:14:37
21347,21348,2015-12-31,21:23:10
21348,21349,2015-12-31,22:09:54


In [ ]:
# prompt: check if there null value in all exel sheet

sheets_dict = pd.read_excel(io.BytesIO(uploaded['pizza_sale.xlsx']), sheet_name=None)

for sheet_name, df in sheets_dict.items():
    print(f"Checking sheet: {sheet_name}")
    if df.isnull().values.any():
        print(f"  Null values found in sheet '{sheet_name}':")
        print(df[df.isnull().any(axis=1)])
    else:
        print(f"  No null values found in sheet '{sheet_name}'.")



Checking sheet: order_details
  No null values found in sheet 'order_details'.
Checking sheet: orders
  No null values found in sheet 'orders'.
Checking sheet: pizza type
  No null values found in sheet 'pizza type'.
Checking sheet: pizzas
  No null values found in sheet 'pizzas'.
Checking sheet: data_dictionary
  No null values found in sheet 'data_dictionary'.


In [ ]:
# prompt: add new column in "order_details" named "order_price" at column E
# by in this column it will check value in column C "pizza_id" in sheet "order_details" and match it with "pizza_id" in "pizzas" sheet column A then it will return value in column D "price" in "pizzas" sheet and then after that multiplie it with value from column D "quantity" in order_details" sheet

# Load the order_details and pizzas sheets
order_details_df = sheets_dict['order_details']
pizzas_df = sheets_dict['pizzas']

# Merge order_details and pizzas DataFrames on 'pizza_id'
merged_df = pd.merge(order_details_df, pizzas_df[['pizza_id', 'price']], on='pizza_id', how='left')

# Calculate the order_price and insert it at column E
# Get the current columns of order_details_df
cols = order_details_df.columns.tolist()

# Find the index for column E (which is index 4)
insert_index = 4

# Calculate the 'order_price'
merged_df['order_price'] = merged_df['quantity'] * merged_df['price']

# Extract the calculated 'order_price' column
order_price_col = merged_df['order_price']

# Insert the 'order_price' column into the original order_details_df
order_details_df.insert(insert_index, 'order_price', order_price_col)

# Display the updated DataFrame
print(order_details_df.head())


   order_details_id  order_id       pizza_id  quantity  order_price
0                 1         1     hawaiian_m         1        13.25
1                 2         2  classic_dlx_m         1        16.00
2                 3         2  five_cheese_l         1        18.50
3                 4         2    ital_supr_l         1        20.75
4                 5         2     mexicana_m         1        16.00


In [ ]:
# prompt: make new column in "orders" sheet named "bill_total" where it calculate total from checking value from "order_id" column B in "order_details" sheet if it the same number it in the same bill so keep adding value from column E "order_price" untill it found a new value in "order_id" row then reset as a new bill

orders_df = sheets_dict['orders']

# Initialize the 'bill_total' column with zeros
orders_df['bill_total'] = 0.0

current_order_id = None
current_bill_total = 0.0
orders_df_index = 0

for index, row in order_details_df.iterrows():
    order_id = row['order_id']
    order_price = row['order_price']

    if current_order_id is None or order_id == current_order_id:
        current_bill_total += order_price
        current_order_id = order_id
    else:
        # Found a new order_id, update the bill_total for the previous order_id
        # Find the row in orders_df with the previous current_order_id
        while orders_df_index < len(orders_df) and orders_df.loc[orders_df_index, 'order_id'] != current_order_id:
            orders_df_index += 1

        if orders_df_index < len(orders_df) and orders_df.loc[orders_df_index, 'order_id'] == current_order_id:
            orders_df.loc[orders_df_index, 'bill_total'] = current_bill_total
            orders_df_index += 1 # Move to the next potential row in orders_df

        # Reset for the new order_id
        current_order_id = order_id
        current_bill_total = order_price

# After the loop, update the bill_total for the very last order_id
if current_order_id is not None:
    while orders_df_index < len(orders_df) and orders_df.loc[orders_df_index, 'order_id'] != current_order_id:
        orders_df_index += 1

    if orders_df_index < len(orders_df) and orders_df.loc[orders_df_index, 'order_id'] == current_order_id:
        orders_df.loc[orders_df_index, 'bill_total'] = current_bill_total


# Display the updated orders_df with the new 'bill_total' column
print(orders_df.head())


   order_id       date      time  bill_total
0         1 2015-01-01  11:38:36       13.25
1         2 2015-01-01  11:57:40       92.00
2         3 2015-01-01  12:12:28       37.25
3         4 2015-01-01  12:16:31       16.50
4         5 2015-01-01  12:21:30       16.50


In [ ]:
# prompt: make a new sheet named "monthly_sale" that have column as followed "month","total_sale","total_bill"

# Ensure 'date' column is in datetime format
orders_df['date'] = pd.to_datetime(orders_df['date'])

# Extract month and year and create a 'month' column
orders_df['month'] = orders_df['date'].dt.to_period('M')

# Group by the newly created 'month' column and sum 'bill_total'
monthly_sales_df = orders_df.groupby('month')['bill_total'].sum().reset_index()

# Rename the columns to match the required names
monthly_sales_df.rename(columns={'month': 'month', 'bill_total': 'total_sale'}, inplace=True)

# Calculate the total number of bills per month
monthly_bills_df = orders_df.groupby('month').size().reset_index(name='total_bill')

# Merge the monthly sales and monthly bills DataFrames
monthly_sale_df = pd.merge(monthly_sales_df, monthly_bills_df, on='month')

# Reorder columns to "month", "total_sale", "total_bill"
monthly_sale_df = monthly_sale_df[['month', 'total_sale', 'total_bill']]

# Display the new DataFrame
print(monthly_sale_df.head())

     month  total_sale  total_bill
0  2015-01    69793.30        1845
1  2015-02    65159.60        1685
2  2015-03    70397.10        1840
3  2015-04    68736.80        1799
4  2015-05    71402.75        1853


In [ ]:
# prompt: save all the changed into original file

# Prepare the sheets dictionary to be written back to the Excel file
output_sheets = sheets_dict.copy()
output_sheets['order_details'] = order_details_df
output_sheets['orders'] = orders_df
output_sheets['monthly_sale'] = monthly_sale_df # Add the new sheet

# Define the output filename
output_filename = 'pizza_sale_updated.xlsx'

# Create an Excel writer object
with pd.ExcelWriter(output_filename) as writer:
    for sheet_name, df_to_save in output_sheets.items():
        # Convert 'month' column to string if it exists and is Period type
        if 'month' in df_to_save.columns and isinstance(df_to_save['month'].dtype, pd.core.dtypes.dtypes.PeriodDtype):
            df_to_save['month'] = df_to_save['month'].astype(str)

        df_to_save.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Updated data saved to '{output_filename}'")

# Optionally, download the updated file
files.download(output_filename)

Updated data saved to 'pizza_sale_updated.xlsx'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# prompt: make a new sheet name "price_sale" the first column name "price" and the next column name "quantity" that it count all the pizza with that prize by checking pizza_id in "order_details" then check their price from "pizzas" sheet then check how much they order in "quantity" column in "order_details" sheet

# Group by 'price' and count the occurrences of each price in the merged DataFrame
price_quantity_df = merged_df.groupby('price')['quantity'].sum().reset_index()

# Rename columns to 'price' and 'quantity'
price_quantity_df.rename(columns={'quantity': 'quantity'}, inplace=True)

# Create the 'price_sale' sheet DataFrame
price_sale_df = price_quantity_df[['price', 'quantity']]

# Display the new DataFrame
print(price_sale_df.head())

# Add the new sheet to the dictionary of sheets
output_sheets['price_sale'] = price_sale_df

# Define the output filename
output_filename = 'pizza_sale_updated.xlsx'

# Create an Excel writer object
with pd.ExcelWriter(output_filename) as writer:
    for sheet_name, df_to_save in output_sheets.items():
        # Convert 'month' column to string if it exists and is Period type
        if 'month' in df_to_save.columns and isinstance(df_to_save['month'].dtype, pd.core.dtypes.dtypes.PeriodDtype):
            df_to_save['month'] = df_to_save['month'].astype(str)

        df_to_save.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Updated data saved to '{output_filename}'")

# Optionally, download the updated file
files.download(output_filename)

   price  quantity
0   9.75       751
1  10.50      1020
2  11.00       578
3  12.00      5744
4  12.25       850
Updated data saved to 'pizza_sale_updated.xlsx'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>